In [ ]:
info_theory = {"camb": {"extra_args": {"lens_potential_accuracy": 1}}}
#info_theory = {"classy": {"extra_args": {"non linear": "halofit"}}}

# Path to installation
modules = ""  ## SET!!!

# Number of runs over which to average
n = 200

In [ ]:
import os

from cobaya.conventions import _likelihood, _theory, _timing, _path_install, _params
from cobaya.tools import recursive_update, get_modules
from cobaya.cosmo_input import create_input, planck_base_model
from cobaya.model import get_model

params_test = {
    "omegabh2": 0.022383,
    "omegach2": 0.12011,
    "H0": 67.32,
    "logA": 3.0448,
    "ns": 0.96605,
    "tau": 0.0543}

info = create_input(planck_names=True, theory=list(info_theory)[0], **planck_base_model)
info[_params]["H0"] = {"prior": {"min": 0, "max": 100}}
for p in list(info[_params]):
    if p.startswith("theta") or p.startswith("cosmomc"):
        info[_params].pop(p)
info = recursive_update(info, {_theory: info_theory})

# Constrain to only:
only = []
only_startswith = []
only_contains = []

# Exclude
non_startswith = [
    # Ignore non-cosmo and generic likes
    "one", "gaussian_mixture", "_", ".",
]
non_contains = [
    # EXCLUDE EITHER Planck 2015 or 2018 (memory errors)
    "planck_2015",
    # EXCLUDE EITHER Planck 2018 CamSpec or plik (incompatible priors and parameter names)
    "plik",
]

# For Planck, select either TT or TTTEEE
select = "TT"
if select == "TT":
    non_contains += ["TTTEEE"]
elif select == "TTTEEE":
    only_contains += ["TTTEEE"]

# NB: It's possible that DES makes everything else slower, since it adds many more
# redshifts to the computation, so there are many more redshifts to select the
# required e.g. H(z) from, making e.g. SN and BAO likes slower (longer theory code request)
#non_contains += ["des"]

likelihoods = sorted([
    like for like in get_modules("likelihood") if
    (any([like.startswith(s) for s in only_startswith]) if only_startswith else True) and 
    not any([like.startswith(s) for s in non_startswith]) and
    (any([(s in like) for s in only_contains]) if only_contains else True) and
    not any([(s in like) for s in non_contains]) and
    (like in only if only else True)])

info[_likelihood] = {l:None for l in likelihoods}

info["debug"] = False

# No fsigma8 in classy!
if list(info_theory)[0] == "classy":
    info[_likelihood].pop("sdss_dr12_consensus_final")
    info[_likelihood].pop("sdss_dr12_consensus_full_shape")

print("\n".join(likelihoods))

In [ ]:
info[_timing] = True
info[_path_install] = modules
model = get_model(info)

In [ ]:
for i in range(n):
    point = dict(zip(model.parameterization.sampled_params(), model.prior.sample(ignore_external=True)[0]))
    point.update(params_test)
    model.loglike(point, cached=False)
    print("%d%%"%(100*i/n), end=" ")
print("done!")

In [ ]:
from pandas import DataFrame
df = DataFrame(columns=["time", "time_std", "speed-", "speed", "speed+", "old speed"], index=likelihoods)
for name in [_theory] + list(model.likelihood):
    like = model.likelihood[name]
    df.at[name, "time"] = like.time_avg
    df.at[name, "time_std"] = like.time_std
    df.at[name, "speed-"] = 1/(like.time_avg+like.time_std) if like.time_std else 0
    df.at[name, "speed"] = 1/like.time_avg
    df.at[name, "speed+"] = 1/(like.time_avg-like.time_std) if like.time_std else 0
    df.at[name, "time_std"] = like.time_std
    df.at[name, "old speed"] = like.speed
    df.at[name, "factor"] = like.speed*like.time_avg
print(df)